# Focal event finder

Given a cell type and event type, this notebook will iterate through all of Giulia's experiments, finding the corresponding time points of said events

In [132]:
import os
from natsort import natsorted
import sys
sys.path.append('../')
import dataio
from collections import defaultdict
import numpy as np
import json
from tqdm import tqdm
import glob

In [2]:
cell_type = 'Scr'
event_type = 'div'
root_dir = '/home/nathan/data/kraken/h2b/giulia/'
root_dir_server = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/'

# Get Chris' first
Chris has already made glimpses for scr division

Get list of experiments canonical list

In [ ]:
### canonical 'pure' competition experiments from '/home/nathan/data/kraken/h2b/giulia/canon_expts.json'
expt_dict

In [203]:
scr_divs = defaultdict(dict)
error_mito = []
for expt in tqdm(expt_dict):
    pos_list = expt_dict[expt]
    for pos in pos_list:
        glimpse_dir = os.path.join(root_dir_server, expt, pos, pos+'_aligned/glimpses/mit_glimpses')
        mito_glimpse_fns = glob.glob(os.path.join(glimpse_dir, '*RFP_mit.npz')) #[fn for fn in os.listdir(glimpse_dir) if 'RFP_mit.npz' in fn]
        if mito_glimpse_fns == []:
            scr_divs[expt][pos] = 'No mitoses recorded'
        else:
            ### create empty list for mito info storage
            mito_list = []
            ### iterate through all mitoses in expt pos
            for mito_fn in mito_glimpse_fns:
                try:
                    ### load mito information
                    mito_npz = np.load(mito_fn)
                    ### get cell ID of mitosis
                    ID = int(mito_npz['ID'])
                    ### get the time of division
                    t_div = int(mito_npz['cutoff']) 
                    ### check if t_div is in glimpse time range
                    if t_div not in list(mito_npz['t']):
                        ### if not then append to error log and record empty xy
                        error_mito.append((expt, pos, ID, 'time error'))
                        x_div, y_div = '', ''
                        mito_list.append((ID, t_div, x_div, y_div))
                        continue
                    else:
                        ### get the index of that frame in the list of timepoints
                        index_div = list(mito_npz['t']).index(t_div)
                        ### use that index to find x and y coord
                        x_div = list(mito_npz['x'])[index_div]
                        y_div = list(mito_npz['y'])[index_div]
                        ### save mitosis information as tuple in mutlidim dict
                        mito_list.append((ID, t_div, x_div, y_div))
                except:
                    error_mito.append((expt, pos, ID, 'unknown error'))
                    
            scr_divs[expt][pos] = mito_list


100%|██████████| 20/20 [00:38<00:00,  1.92s/it]


In [204]:
error_mito

[('GV0794', 'Pos5', 430, 'time error'),
 ('GV0794', 'Pos6', 605, 'time error'),
 ('GV0795', 'Pos3', 170, 'time error'),
 ('GV0795', 'Pos3', 265, 'time error'),
 ('GV0795', 'Pos4', 318, 'time error'),
 ('GV0795', 'Pos4', 436, 'time error'),
 ('GV0795', 'Pos4', 398, 'time error'),
 ('GV0795', 'Pos5', 319, 'time error'),
 ('GV0795', 'Pos5', 620, 'time error'),
 ('GV0795', 'Pos5', 1271, 'time error'),
 ('GV0795', 'Pos5', 619, 'time error'),
 ('GV0795', 'Pos5', 614, 'time error'),
 ('GV0795', 'Pos5', 838, 'time error'),
 ('GV0795', 'Pos5', 579, 'time error'),
 ('GV0795', 'Pos5', 381, 'time error'),
 ('GV0795', 'Pos5', 277, 'time error'),
 ('GV0795', 'Pos5', 747, 'time error'),
 ('GV0795', 'Pos5', 535, 'time error'),
 ('GV0795', 'Pos6', 1158, 'time error'),
 ('GV0795', 'Pos6', 813, 'time error'),
 ('GV0795', 'Pos6', 374, 'time error'),
 ('GV0795', 'Pos7', 718, 'time error'),
 ('GV0795', 'Pos7', 260, 'time error'),
 ('GV0795', 'Pos7', 252, 'time error'),
 ('GV0795', 'Pos7', 717, 'time error')

In [208]:
scr_div_c

defaultdict(dict,
            {'GV0794': {'Pos4': 'No mitoses recorded',
              'Pos5': [(15, 323, 1079.5496826171875, 1091.0084228515625),
               (6, 132, 893.743408203125, 394.80914306640625),
               (430, 92, '', ''),
               (41, 112, 197.9872589111328, 1081.90869140625),
               (7, 505, 1006.2675170898438, 524.1834106445312)],
              'Pos6': [(605, 131, '', ''),
               (52, 57, 364.52996826171875, 1117.410888671875)],
              'Pos7': [(60, 548, 220.1459197998047, 682.2733764648438),
               (300, 498, 522.7280883789062, 903.0943603515625),
               (58, 301, 323.101318359375, 656.7352294921875),
               (294, 449, 593.8465576171875, 757.8079223632812),
               (56, 378, 488.0382995605469, 987.0674438476562),
               (293, 510, 637.0244140625, 672.5520629882812)],
              'Pos8': 'No mitoses recorded'},
             'GV0795': {'Pos3': [(170, 447, '', ''),
               (46, 535, 1053

# Troubleshooting  ('GV0819', 'Pos2', 29, 'unknown error'),


In [206]:
pos = 'Pos2'
expt = 'GV0819'

In [210]:
mito_glimpse_fns

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_Pos2_1448_RFP_mit.npz',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_Pos2_91_RFP_mit.npz',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_Pos2_1132_RFP_mit.npz',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_Pos2_1261_RFP_mit.npz',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_Pos2_454_RFP_mit.npz',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0819/Pos2/Pos2_aligned/glimpses/mit_glimpses/GV0819_P

In [265]:
scr_divs = defaultdict(dict)
error_mito = []
zero_confidence_count, med_confidence_count, low_confidence_count, high_confidence_count, = 0,0,0,0
for expt in tqdm(expt_dict):
    pos_list = expt_dict[expt]
    for pos in pos_list:
        glimpse_dir = os.path.join(root_dir_server, expt, pos, pos+'_aligned/glimpses/mit_glimpses')
        mito_glimpse_fns = glob.glob(os.path.join(glimpse_dir, '*RFP_mit.npz')) #[fn for fn in os.listdir(glimpse_dir) if 'RFP_mit.npz' in fn]
        if mito_glimpse_fns == []:
            scr_divs[expt][pos] = 'No mitoses recorded'
        else:
            ### create empty list for mito info storage
            mito_list = []
            ### iterate through all mitoses in expt pos
            for mito_fn in mito_glimpse_fns:
                ### load mito information
                mito_npz = np.load(mito_fn)
                ### try get cell ID of mitosis, if not then record error
                try:
                    ID = int(mito_npz['ID'])
                except:
                    error_mito.append((expt, pos, mito_fn.split('_')[2], 'ID loading error'))
                    continue
                ### try get the time of division, if not try using last timepoint and record as unconfident
                try:
                    t_div = int(mito_npz['cutoff']) 
                except:
                    try:
                        t_div = int(list(mito_npz['t'])[-1])
                        x_div = int(list(mito_npz['x'])[-1])
                        y_div = int(list(mito_npz['y'])[-1])
                        mito_list.append((ID, t_div, x_div, y_div, 'low confidence', 'no cut off time'))
                        low_confidence_count+=1
                        continue 
                    except:
                        error_mito.append((expt, pos, ID, 'Cut off time error'))
                        zero_confidence_count+=1
                        continue
                ### check if t_div is in glimpse time range
                if t_div not in list(mito_npz['t']):
                    ### if not then append to error log and record empty xy
                    error_mito.append((expt, pos, ID, 'time error'))
                    x_div, y_div = '', ''
                    mito_list.append((ID, t_div, x_div, y_div,'medium confidence', 't_div not in t range'))
                    med_confidence_count +=1
                    continue
                else:
                    ### get the index of that frame in the list of timepoints
                    index_div = int(list(mito_npz['t']).index(t_div))
                    ### use that index to find x and y coord
                    x_div = int(list(mito_npz['x'])[index_div])
                    y_div = int(list(mito_npz['y'])[index_div])
                    ### save mitosis information as tuple in mutlidim dict
                    mito_list.append((ID, t_div, x_div, y_div,'high confidence', 't_div present and in t range'))
                    high_confidence_count+=1
            scr_divs[expt][pos] = mito_list


100%|██████████| 20/20 [00:38<00:00,  1.94s/it]


In [266]:
scr_divs

defaultdict(dict,
            {'GV0794': {'Pos4': 'No mitoses recorded',
              'Pos5': [(15,
                323,
                1079,
                1091,
                'high confidence',
                't_div present and in t range'),
               (6,
                132,
                893,
                394,
                'high confidence',
                't_div present and in t range'),
               (430, 92, '', '', 'medium confidence', 't_div not in t range'),
               (41,
                112,
                197,
                1081,
                'high confidence',
                't_div present and in t range'),
               (7,
                505,
                1006,
                524,
                'high confidence',
                't_div present and in t range')],
              'Pos6': [(605,
                131,
                '',
                '',
                'medium confidence',
                't_div not in t range'),
 

In [267]:
zero_confidence_count, low_confidence_count, med_confidence_count, high_confidence_count

(0, 121, 385, 748)

In [268]:
0+121+385+748

1254

In [271]:
with open('/home/nathan/data/kraken/h2b/giulia/experiment_information/divisions/scr_div_info.json', 'w') as file:
    json.dump(scr_divs, file)

# Making new division dicts for radial analysis generation from previous div info dicts

In [272]:
scr_divs_c = scr_divs

In [286]:
scr_div_dict = {}
for expt in scr_divs_c:
    pos_list = scr_divs_c[expt]
    for pos in pos_list:
        if scr_divs_c[expt][pos] == 'No mitoses recorded':
            continue
        else:
            for mitosis in scr_divs_c[expt][pos]:
                if mitosis[4] == 'high confidence':
                    ID = '{}_{}_{}_RFP'.format(expt, pos, mitosis[0])
                    scr_div_dict[ID] = mitosis[1]

In [287]:
scr_div_dict

{'GV0794_Pos5_15_RFP': 323,
 'GV0794_Pos5_6_RFP': 132,
 'GV0794_Pos5_41_RFP': 112,
 'GV0794_Pos5_7_RFP': 505,
 'GV0794_Pos6_52_RFP': 57,
 'GV0794_Pos7_60_RFP': 548,
 'GV0794_Pos7_300_RFP': 498,
 'GV0794_Pos7_58_RFP': 301,
 'GV0794_Pos7_294_RFP': 449,
 'GV0794_Pos7_56_RFP': 378,
 'GV0794_Pos7_293_RFP': 510,
 'GV0795_Pos3_46_RFP': 535,
 'GV0795_Pos4_36_RFP': 99,
 'GV0795_Pos4_30_RFP': 407,
 'GV0795_Pos4_12_RFP': 67,
 'GV0795_Pos4_19_RFP': 394,
 'GV0795_Pos4_24_RFP': 92,
 'GV0795_Pos5_164_RFP': 285,
 'GV0795_Pos5_14_RFP': 507,
 'GV0795_Pos5_376_RFP': 472,
 'GV0795_Pos5_50_RFP': 476,
 'GV0795_Pos5_52_RFP': 395,
 'GV0795_Pos5_37_RFP': 351,
 'GV0795_Pos5_19_RFP': 120,
 'GV0795_Pos5_169_RFP': 370,
 'GV0795_Pos5_6_RFP': 449,
 'GV0795_Pos5_29_RFP': 775,
 'GV0795_Pos5_13_RFP': 438,
 'GV0795_Pos5_39_RFP': 363,
 'GV0795_Pos6_150_RFP': 572,
 'GV0795_Pos6_16_RFP': 46,
 'GV0795_Pos6_35_RFP': 416,
 'GV0795_Pos6_151_RFP': 284,
 'GV0795_Pos6_164_RFP': 442,
 'GV0795_Pos6_377_RFP': 414,
 'GV0795_Pos6_444_

In [288]:
len(scr_div_dict)

748

In [289]:
with open('/home/nathan/data/kraken/h2b/giulia/experiment_information/divisions/scr_div_dict_confident.json', 'w') as file:
    json.dump(scr_div_dict, file)

# Generating own glimpses

This method is problematic as the previously generated tracking files frequently mistake a scribble apoptosis for a division

Need to filter list of experiments and positions to only include relevant ones

In [103]:
### canonical 'pure' competition experiments from '/home/nathan/data/kraken/h2b/giulia/canon_expts.json'
expt_dict

defaultdict(list,
            {'GV0794': ['Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8'],
             'GV0795': ['Pos3',
              'Pos4',
              'Pos5',
              'Pos6',
              'Pos7',
              'Pos8',
              'Pos16',
              'Pos17',
              'Pos18'],
             'GV0796': ['Pos3',
              'Pos4',
              'Pos5',
              'Pos6',
              'Pos7',
              'Pos8',
              'Pos19',
              'Pos20',
              'Pos21',
              'Pos22',
              'Pos23',
              'Pos24'],
             'GV0797': ['Pos3',
              'Pos4',
              'Pos5',
              'Pos6',
              'Pos7',
              'Pos8',
              'Pos18',
              'Pos19',
              'Pos20',
              'Pos21',
              'Pos22',
              'Pos23'],
             'GV0798': ['Pos4',
              'Pos5',
              'Pos6',
              'Pos7',
              'Pos8',
              'Pos18',


In [125]:
scr_divs = defaultdict(dict)
for expt in tqdm(expt_dict):
    pos_list = expt_dict[expt]
    #pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_dir, expt_list[0])) if 'Pos' in pos])
    for pos in pos_list:
        track_fn = os.path.join(root_dir, expt, pos, pos+'_aligned', 'HDF/segmented.hdf5')
        ### load track
        try:
            wt_cells, scr_cells, all_cells = dataio.load_tracking_data(track_fn)
            print(expt, pos, 'success')
            #wt_tracks, scr_tracks, wt_data, scr_data, shift_x, shift_y = dataio.load_and_align_tracks(track_fn, np.zeros((10,10,10)))
            ### find scr divisions
            list_of_divisions = [(cell.ID, cell.t[-1], cell.x[-1], cell.y[-1]) for cell in scr_cells if cell.fate.name == 'DIVIDE']
            ### create dict storage of scr divisions
            scr_divs[expt][pos] = list_of_divisions
        except: 
            print(expt, pos, 'could not load')


  0%|          | 0/20 [00:00<?, ?it/s][INFO][2021/09/02 03:10:16 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:10:16 PM] Loading tracks/obj_type_1


Btrack version no.: 0.4.2


[INFO][2021/09/02 03:10:16 PM] Loading objects/obj_type_1 (69360, 5) (64986 filtered: area>=100)
[INFO][2021/09/02 03:10:17 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:10:17 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:10:17 PM] Loading tracks/obj_type_2
[INFO][2021/09/02 03:10:17 PM] Loading objects/obj_type_2 (13865, 5) (10538 filtered: area>=100)
[INFO][2021/09/02 03:10:17 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:10:17 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:10:17 PM] Loading tracks/obj_type_1


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
GV0794 Pos4 success
Btrack version no.: 0.4.2


[INFO][2021/09/02 03:10:17 PM] Loading objects/obj_type_1 (31315, 5) (20377 filtered: area>=100)
[INFO][2021/09/02 03:10:18 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:10:18 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:10:18 PM] Loading tracks/obj_type_2
[INFO][2021/09/02 03:10:18 PM] Loading objects/obj_type_2 (61992, 5) (46754 filtered: area>=100)
[INFO][2021/09/02 03:10:19 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:10:19 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos6/Pos6_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:10:19 PM] Loading tracks/obj_type_1
[INFO][2021/09/02 03:10:19 PM] Loading objects/obj_type_1 (13419, 5) (11379 filtered: area>=100)


...

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
GV0820 Pos3 success
Btrack version no.: 0.4.2


[INFO][2021/09/02 03:22:02 PM] Loading objects/obj_type_1 (189653, 5) (175798 filtered: area>=100)
[INFO][2021/09/02 03:22:05 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:22:05 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:22:05 PM] Loading tracks/obj_type_2
[INFO][2021/09/02 03:22:05 PM] Loading objects/obj_type_2 (82474, 5) (72948 filtered: area>=100)
[INFO][2021/09/02 03:22:06 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:22:06 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:22:06 PM] Loading tracks/obj_type_1


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
GV0820 Pos4 success
Btrack version no.: 0.4.2


[INFO][2021/09/02 03:22:07 PM] Loading objects/obj_type_1 (168718, 5) (157112 filtered: area>=100)
[INFO][2021/09/02 03:22:10 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 03:22:10 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 03:22:10 PM] Loading tracks/obj_type_2
[INFO][2021/09/02 03:22:11 PM] Loading objects/obj_type_2 (75424, 5) (61898 filtered: area>=100)
[INFO][2021/09/02 03:22:11 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0820/Pos5/Pos5_aligned/HDF/segmented.hdf5
100%|██████████| 20/20 [11:55<00:00, 35.80s/it]

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
GV0820 Pos5 success


In [128]:
scr_divs_auto = scr_divs

In [127]:
scr_divs['GV0795']['Pos5']

[(-37, 421, 198.7002716064453, 559.3937377929688),
 (-38, 158, 261.57440185546875, 793.9803466796875),
 (-39, 390, 322.3199157714844, 583.4186401367188),
 (-2, 996, 678.1123046875, 329.11163330078125),
 (-47, 370, 647.772705078125, 813.0022583007812),
 (-52, 407, 471.4093322753906, 478.79595947265625),
 (-29, 789, 820.1409912109375, 321.91717529296875),
 (-4, 21, 767.3869018554688, 878.5217895507812),
 (-7, 476, 973.8013305664062, 1475.622314453125),
 (-13, 460, 923.3218994140625, 307.3121032714844),
 (-14, 517, 1091.75830078125, 112.24861907958984),
 (-22, 47, 1068.9276123046875, 1140.2637939453125),
 (-18, 365, 957.05810546875, 1476.5689697265625),
 (-20, 346, 996.8843383789062, 635.4411010742188),
 (-19, 131, 1032.22509765625, 643.2670288085938),
 (-107, 66, 1062.6693115234375, 1193.991943359375),
 (-160, 123, 855.6416015625, 1055.87548828125),
 (-164, 418, 787.7554321289062, 780.5281372070312),
 (-169, 509, 382.0804138183594, 1085.254638671875),
 (-172, 880, 711.3300170898438, 669.